In [28]:
#创建数据库test
import pymysql

conn = pymysql.connect(  # 连接本地数据库
    host="localhost",
    user="root",
    password="123456", 
    charset="utf8"
)

cursor = conn.cursor()
cursor .execute("create database test character set utf8;")
# 执行完之后关闭游标和数据库连接
cursor.close()
conn.close()

#### 列属性
真正约束一个字段的是数据类型，但是数据类型的约束很单一，有时候还需要有一些额外的约束，这些额外的约束就是列属性。
列属性主要有:null/not null，default，primary key, unique key, auto_increment, comment等
##### 空属性
空属性有两个值：NULL(默认) NOT NULL(不为空)
虽然是默认的，数据库基本都是字段为空，但是实际上在真实数据库设计的时候，尽可能的要保证所有的数据都不应该为空，因为空数据没有意义和空数据没有办法参与运算。
##### 列描述：comment
描述：comment，没有实际含义，是专门用来描述字段，会根据表创建语句保存，用来给数据库管理员来进行了解的。例如新来一个运维或者开发接手项目，新来的可以通过commment的内容来接当前列字段是什么业务数据。
##### 默认值：关键字default
默认值就是:某一种数据会经常性的出现某个具体的值。可以在一开始就制定好，在需要真实数据的时候，用户可以选择性的使用默认值。
#####  zerofill
 zerofull又称零填充，只能作用在无符号数据类型上，会使数据省略的空位用0来填充，对数据大小没有任何改变，只是方便观察数据位数。
 比如当我们设置一个无符号的int类型字段，对该字段插入一个 10，而一个无符号int可以存储~294967295，
##### 主键primary key
主键 : primary key用来唯一的约束该字段里面的数据,不能重复,不能为空,一张表最多只能有一个主键, 主键对应的类通常是整数类型
创建表的时候直接在字段上指定主键
##### 自增长auto_increment
auto_increment : 当对应的字段, 不给值,会自动被系统触发,系统会从当前字段中已经有的最大值 + 1 操作,得到一个新的不同的值,***经常和主键搭配使用,作为逻辑主键***
自增长的特点
任何一个字段要做自增长,前提是本身是一个索引(key 一栏有值)
自增字段必须是整数
一张表最多只有一个自增长
##### 唯一键
当表中需要除主键外还需字段唯一,可使用唯一键,即: 唯一键可解决表中多个字段需要唯一性约束
##### 外键
外键用于定义主表和从表之间的关系：外键约束主要定义在从表上，主表则必须是有主键约束或unique约束。当定义外键后，要求外键列数据必须在主表的主键列存在或为null。

use test #选择test库
create table 表名(    #创建表
    列名  类型  是否可以为空，
    列名  类型  是否可以为空
) 

In [29]:
#创建表
import pymysql 
conn = pymysql.connect(  # 连接本地数据库
    host="localhost",
    user="root",
    password="123456",
    database = "test",
    charset="utf8"
)
 # 使用 cursor() 方法创建一个游标对象 cursor
cursor = conn.cursor() 

sql = """
create table information(
    id int auto_increment primary key ,
    name VARCHAR(255) not null,
    job VARCHAR(255) not null,
    age TINYINT NOT NULL 
    );
"""

try:
    cursor.execute(sql)# 使用游标执行sql，#使用 execute() 方法执行 sql 查询
except Exception as e:
    print(e)
    # 发生错误则回滚
    conn.rollback()

回滚操作都是写在异常处理，或是sql的最后。如果sql中出现错误，代码会立即跳转到错误处理代码上执行，比如回滚，但紧接在错误行之后的代码不会执行的。暂时还不是很明白回滚。
如果对一个或多个更改不满意，并且要完全还原这些更改，用rollback()方法。

In [30]:
#一次插入多条数据
data = [
    ('张三', '老师',33),
    ('李四', '学生',20),
    ('王五', '程序员',30),
]

try:
    # 执行sql语句
   cursor.executemany('insert into information(name,job,age) values(%s, %s, %s);', data)
    # 提交执行
   conn.commit()
except Exception as e:
    print(e)
    # 发生错误则回滚
    conn.rollback()

##### 插入一条数据：
第一种写法：先用字符串利用%s占位生成一条完整的sql语句，然后去执行。
第二种写法：pymysql的游标也自带这一功能，而且看起来更加方便，这种方法的好处是sql语句中的%s不需要加引号
（如果加了引号，引号也会被当做数据插入到数据表中），游标执行这一语句时，会根据数据类型来判断是否要加上引号。

In [31]:
#一次插入一条数据
try:
    # 执行sql语句
    cursor.execute('insert into information(name,job,age) values("%s", "%s", %s);' % ('李六', '律师',28))
    cursor.execute('insert into information(name,job,age) values(%s, %s, %s);', ('李六', '律师',28))
    conn.commit()
except Exception as e:
    print(e)
    # 发生错误则回滚
    conn.rollback()

In [32]:
#更新数据库
try:
    # 执行sql语句
    cursor.execute('update information set job=%s where name=%s;', ["医生", "李六"])
    cursor.execute('update information set name=%s where id=%s;', ["赵七", "5"])
    conn.commit()
except Exception as e:
    print(e)
    # 发生错误则回滚
    conn.rollback()

插入、更新、删除操作必须再执行commit操作之后才会生效，而查询操作也只有在执行fetch操作之后才会生效。
fetch操作包括3个方法，分别是fetchone()、fetchall()、fetchmany()。

In [33]:
try:
   #fetchall()：查询指定的所有记录
    cursor.execute('select * from information where id < %s;', [6])
    result1 = cursor.fetchall()
    print(result1)

    #fetchmany(size)：查询指定数量的记录,取出符合查询条件的最前面的指定数量记录。这里的size指的就是想要取出的记录条数。
    cursor.execute('select * from information where id < %s;', [4])
    result2 = cursor.fetchmany(2)
    print(result2)

    #fetchone()：取出第一条记录
    cursor.execute('select * from information where id < %s;', [4])
    result3 = cursor.fetchone()
    print(result3)
except Exception as e:
    print(e)
    print ("Error: unable to fetch data")

((1, '张三', '老师', 33), (2, '李四', '学生', 20), (3, '王五', '程序员', 30), (4, '李六', '医生', 28), (5, '赵七', '医生', 28))
((1, '张三', '老师', 33), (2, '李四', '学生', 20))
(1, '张三', '老师', 33)


首先，order by是用来写在where之后，给多个字段来排序的一个DQL查询语句。
其次，order by写法：
1.  select 字段列表/* from 表名 where 条件 order by 字段名1 asc/desc, 字段名2 asc/desc,.......
2.  select 字段列表/* from 表名 where 条件 order by 字段序号 asc/desc, 字段序号 asc/desc,....... （此时字段序号要从1开始）
3.  select 字段列表/* from 表名 where 条件 order by 字段别名1 asc/desc, 字段别名2 asc/desc,.......（这里类似于第一种，无非就是把字段名加了个别名来代替而已。
然后，order by的方式：
1.asc 升序，可以省略，是数据库默认的排序方式
2.desc 降序，跟升序相反。
最后要注意order by的原则，写在最前面的字段，他的优先级最高，也就是写法中第一个的字段名1的优先级最高，优先执行他的内容。
没有结果，再继续学习。

In [34]:
# 排序
#cursor.execute("select * from information order by age asc")

In [35]:
#删除一条
cursor.execute('delete from information where id=%s;', [1])
conn.commit()
#删除多条
cursor.executemany('delete from information where id=%s;', [[2], [4]])
conn.commit()

游标
Cursor返回的数据是以元组的方式保存，
DictCursor是以字典的形式保存，在conn.sursor()方法中传入DictCursor这个类即可。

In [36]:
cursor = conn.cursor(pymysql.cursors.DictCursor) # 创建一个字典游标
cursor.execute('select * from information where id < %s;', [5])
result4 = cursor.fetchall()#所有
print(result4)

cursor.execute('select * from information where id < %s;', [5])
result5 = cursor.fetchone()#第一个
print(result5)

[{'id': 3, 'name': '王五', 'job': '程序员', 'age': 30}]
{'id': 3, 'name': '王五', 'job': '程序员', 'age': 30}


In [37]:
# 执行完之后别忘了关闭游标和数据库连接
cursor.close()
conn.close()
